In [ ]:
import aiohttp
import asyncio
import uuid
import logging
from tqdm import tqdm
import mimetypes
mimetypes.add_type('audio/wav', '.wav') # replace 'audio/x-wav', which is not accepted by the api

logging.basicConfig(format="%(asctime)s - %(levelname)s - %(message)s", datefmt='%H:%M:%S', level=logging.INFO)
logger = logging.getLogger()



endpoint = 'https://api.idg.vnpt.vn/stt-service/v1/grpc/async/standard'
authorization = ''
token_id = ''
token_key = ''
headers_auth = {
    'authorization': authorization,
    'token-id': token_id,
    'token-key': token_key,
}
interval_result_check = 1 # in seconds, should set to 10 or 30 to avoid overloading the server



async def call_api_2_step(session: aiohttp.ClientSession, file_path: str):
    try:
        clientSession = str(uuid.uuid1())
        form_data = aiohttp.FormData({
            'clientSession': clientSession,
        })
        # add file
        content_type, _ = mimetypes.guess_type(file_path)
        form_data.add_field('audioFile', open(file_path,'rb'), content_type=content_type)

        # step 1
        logger.info(f'calling file {file_path} with clientSession {clientSession}')
        async with session.post(endpoint, headers=headers_auth, data=form_data) as response:
            result = await response.json()

        # step 2: check status for result every interval
        while result.get('object',{}).get('status') == 'ACCEPTED':
            logger.info('status == ACCEPTED, processing')
            await asyncio.sleep(interval_result_check)
            result = await check_result(session, clientSession)
        logger.info('status == OK, result is ready')
    except Exception as e:
        # custom your error handling here
        result = {'message': str(e)}
    return result

async def check_result(session: aiohttp.ClientSession, clientSession: str):
    form_data = {'clientSession': clientSession}
    async with session.post(endpoint, headers=headers_auth, data=form_data) as response:
        result = await response.json()
    return result

async def run_async_many_files(wav_files: list[str]):
    async with aiohttp.ClientSession() as session:
        tasks = [call_api_2_step(session, file) for file in wav_files]
        results = await asyncio.gather(*tasks, return_exceptions=True)
    return results

async def run_async_many_files_progress(wav_files: list[str]):
    progress_bar = tqdm(total=len(wav_files))
    async def add_update_progress(task):
        result = await task
        progress_bar.update()
        return result

    async with aiohttp.ClientSession() as session:
        tasks = []
        for file in wav_files:
            task = call_api_2_step(session, file)
            task = add_update_progress(task)
            tasks.append(task)
        results = await asyncio.gather(*tasks, return_exceptions=True)
    return results

In [2]:
file = 'audio/5min-8k.mp3'
results = await run_async_many_files([file])
results

15:28:53 - INFO - calling file audio/5min-8k.mp3 with clientSession 8db2d0a1-ee9b-11ef-9bd0-18c04dba91ec
15:28:54 - INFO - status == ACCEPTED, processing
15:28:55 - INFO - status == OK, result is ready


[{'message': 'IDG-00000000',
  'object': {'results': [{'alternatives': [{'transcript': 'Quay lại đây. Ai ngờ nó quay lại thật tất nhiên, không đội mũ bảo hiểm',
       'confidence': -1.5109931}],
     'channelTag': 1.0}],
   'audio_duration': 4.8,
   'status': 'OK'}}]

In [4]:
logger.setLevel(logging.ERROR)

file = 'audio/3s-8k.wav'
wav_files = [file, file]

results = await run_async_many_files_progress(wav_files)
results

100%|██████████| 2/2 [00:00<00:00,  4.15it/s]


[{'message': 'IDG-00000000',
  'object': {'results': [{'alternatives': [{'transcript': 'Quay lại đây. Ai ngờ nó quay lại thật tất nhiên, không đội mũ bảo hiểm',
       'confidence': -1.5396577}],
     'channelTag': 1.0}],
   'audio_duration': 4.8,
   'status': 'OK'}},
 {'message': 'IDG-00000000',
  'object': {'results': [{'alternatives': [{'transcript': 'Quay lại đây. Ai ngờ nó quay lại thật tất nhiên, không đội mũ bảo hiểm',
       'confidence': -1.5396577}],
     'channelTag': 1.0}],
   'audio_duration': 4.8,
   'status': 'OK'}}]